## 1. Introducción a Spark Streaming
Spark Streaming permite el procesamiento de flujos de datos continuos e ilimitados en tiempo real. A diferencia del procesamiento batch, trabaja con micro-batches de datos, permitiendo procesarlos de forma distribuida.

Esto plantea retos adicionales debido a la impredecibilidad en el ritmo y orden de llegada de los datos. Su uso común incluye cálculos agregados y resúmenes de datos para su consumo por aplicaciones web o motores analíticos. 

Spark Streaming se ha relacionado históricamente con la capa "Speed Layer" de la arquitectura **Lambda** debido a su diseño original, aunque ha evolucionado a un modelo compatible con **Kappa**, con la posibilidad de unificar el procesamiento batch y en streaming en un solo flujo de datos, eliminando la necesidad de dos capas separadas.

Ejemplos de uso en la industria:
- Monitorización de logs de servidores
- Procesamiento de eventos de sensores IoT
- Análisis de transacciones financieras en tiempo real
- Procesamiento de datos de redes sociales

### Streaming en Spark
Spark Streaming es una extensión del núcleo de Spark que permite el procesamiento de flujos de datos en vivo ofreciendo tolerancia a fallos, un alto rendimiento y altamente escalable.

Los datos se pueden ingestar desde diversas fuentes de datos, como Kafka, sockets TCP, etc.. y se pueden procesar mediante funciones de alto nivel, ya sea mediante el uso de RDD y algoritmos MapReduce, o utilizando DataFrames y la sintaxis SQL. 

Finalmente, los datos procesados se almacenan en sistemas de ficheros, bases de datos o cuadros de mandos.

<img src="files/img/03streaming_arch.png" alt="03streaming_arch">

Spark dispone dos soluciones para trabajar con datos en streaming:

- Spark DStream: más antigua, conocida como la primera generación, basada en RDDs
- Spark Structured Streaming basada en el uso de DataFrames y diseñada para construir aplicaciones que puedan reaccionar a los datos en tiempo real.

### DStream
Spark DStream (Discretized Stream) es la primera versión y actualmente no se recomienda su uso.

Funciona mediante un modelo de micro-batching para dividir los flujos de entrada de datos en fragmentos que son procesados por el núcleo de Spark. Este planteamiento tenía mucho sentido cuando el principal modelo de programación de Spark eran los RDD, ya que cada fragmento recibido se representaba mediante un RDD.

Spark DStream recibe los datos de entrada en flujos y los divide en batches, por ejemplo en bloques cada N segundos, los cuales procesa Spark mediante RDD para generar los flujos de resultados procesados:

<img src="files/img/03streaming_flow.png" alt="03streaming_flow">

### Structured Streaming
Spark Structured Streaming es la segunda generación de motor para el tratamiento de datos en streaming, y fue diseñado para ser más rápido, escalable y con mayor tolerancia a los errores que DStream, utilizando el motor de Spark SQL.

Además, podemos expresar los procesos en streaming de la misma manera que realizaríamos un proceso batch con datos estáticos. El motor de Spark SQL se encarga de ejecutar los datos de forma continua e incremental, y actualizar el resultado final como datos streaming. Para ello, podemos utilizar el API de Java, Scala, Python o R para expresar las agregaciones, ventanas de eventos, joins de stream a batch, etc.... Finalmente, el sistema asegura la tolerancia de fallos mediante la entrega de cada mensaje una sola vez (**exactly-once**) a través de checkpoints y logs.

Los pasos esenciales a realizar al codificar una aplicación en streaming son:

1. Especificar uno o más fuentes de datos
2. Desarrollar la lógica para manipular los flujos de entrada de datos mediante transformaciones de DataFrames,
3. Definir el modo de salida
4. Definir el trigger que provoca la lectura
5. Indicar el destino de los datos (data sink) donde escribir los resultados.

<img src="files/img/03streaming_fases.jpg" alt="03streaming_fases">


## 2. Configuración del Entorno
from pyspark.sql import SparkSession

#### Crear una SparkSession con soporte para Streaming
spark = SparkSession.builder \
    .appName("Spark Streaming") \
    .config("spark.sql.streaming.schemaInference", True) \
    .getOrCreate()



## Elementos
La idea básica al trabajar los datos en streaming es similar a tener una tabla de entrada de tamaño ilimitado, y conforme llegan nuevos datos, tratarlos como un nuevo conjunto de filas que se adjuntan a la tabla.

<img src="files/img/03streaming_datatable.png" alt="03streaming_datatable">

### Fuentes de Datos en Structured Streaming
A diferencia del procesamiento batch, donde los datos provienen de fuentes estáticas como archivos en HDFS o S3, en Structured Streaming las fuentes generan datos de forma continua.

Fuentes de datos disponibles:
- Ficheros: Lee archivos en streaming desde un directorio (CSV, JSON, Parquet, etc.).
- Kafka: Consume datos en tiempo real desde un clúster Kafka.
- Socket: Recibe texto en streaming desde una conexión de socket (solo para pruebas).
- Rate: Genera datos sintéticos con timestamps y valores secuenciales (útil para pruebas y benchmarking).
- Tabla (desde Spark 3.1): Lee datos en tiempo real desde una tabla de Spark SQL.


In [0]:
# Simulación de datos en streaming
lineasDF = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

display(lineasDF)

In [0]:
# Simulación de datos en streaming
lineasDF = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# Mostramos resultado agrupado
restoDF = lineasDF.withColumn("resto",col("value") % 10)
cantidadDF = restoDF.groupBy("resto").count()

display(cantidadDF)


In [0]:
# Lee los ficheros que se van añadiendo a un directorio
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql import functions as F

# Esquema ventas
esquemaVentas = StructType([
    StructField("Order ID", IntegerType(), True),
    StructField("Product", StringType(), True),
    StructField("Quantity Ordered", IntegerType(), True),
    StructField("Price Each", DoubleType(), True),
    StructField("Order Date", StringType(), True),
    StructField("Purchase Address", StringType(), True)
])

# Leemos ficheros csv del directorio salesstreaming
dfventas = spark.readStream \
    .option("sep", ",") \
    .schema(esquemaVentas) \
    .csv("dbfs:/FileStore/salesstreaming")

# Crear campos adicionales en el dataframe Ventas: year, month, state, city, CP a partir de los campos Order Date y Purchase Address
dfventas = dfventas.withColumn("timestamp", F.to_timestamp("Order Date","MM/dd/yy HH:mm"))
dfventas = dfventas.withColumn("year", F.year(F.col("timestamp"))).withColumn("month", F.month(F.col("timestamp")))

dfventas = dfventas.withColumn("address", F.split(F.col("Purchase Address"), ","))
dfventas = dfventas.withColumn("state", F.substring(F.col("address")[2],1,3))
dfventas = dfventas.withColumn("CP", F.substring(F.col("address")[2],4,9))
dfventas = dfventas.withColumn("city", F.trim(F.col("address")[1]))

dfventas = dfventas.withColumn("Sales", F.col("Quantity Ordered")*F.col("Price Each"))

# Ciudades con mayores ingresos
dfventasCiudad = dfventas.groupBy(F.col("city")).agg(F.sum(F.col("Sales")).alias("CitySales")).orderBy(F.desc(F.col("CitySales"))) \
    .withColumn("CitySales ($)", format_string("$%,.2f", col("CitySales")))

display(dfventasCiudad.select("city", "CitySales ($)"))



city,CitySales ($)
San Francisco,"$2,490,980.51"
Los Angeles,"$1,611,156.95"
New York City,"$1,381,432.87"
Boston,"$1,070,516.65"
Atlanta,"$841,996.12"
Seattle,"$807,395.08"
Dallas,"$804,232.70"
Portland,"$692,620.55"
Austin,"$523,607.34"
null,$nu


In [0]:
# Crear campos adicionales en el dataframe Ventas: year, month, state, city, CP a partir de los campos Order Date y Purchase Address
dfventas = dfventas.withColumn("timestamp", F.to_timestamp("Order Date","MM/dd/yy HH:mm"))
dfventas = dfventas.withColumn("year", F.year(F.col("timestamp"))).withColumn("month", F.month(F.col("timestamp")))

dfventas = dfventas.withColumn("address", F.split(F.col("Purchase Address"), ","))
dfventas = dfventas.withColumn("state", F.substring(F.col("address")[2],1,3))
dfventas = dfventas.withColumn("CP", F.substring(F.col("address")[2],4,9))
dfventas = dfventas.withColumn("city", F.trim(F.col("address")[1]))

# Obtener el día con mayores ingresos
dfventas = dfventas.withColumn("Sales", F.col("Quantity Ordered")*F.col("Price Each"))
dfventas.groupBy(F.to_date(F.col("timestamp"))).agg(F.sum(F.col("Sales")).alias("DailySales")) \
    .orderBy(F.col("DailySales"),ascending=False).show(3)

# Obtener el producto más vendido (por cantidad total) y qué ingresos ha generado en total.
dfventas.groupBy(F.col("Product")) \
    .agg( \
        F.sum(F.col("Quantity Ordered")).alias("TotalQty"), \
        F.sum(F.col("Sales")) \
    ).orderBy(F.col("TotalQty"),ascending=False).show(3)

# Listar las 10 ciudades con mayores ventas (en ingresos).
dfventas.groupBy(F.col("city")).agg(F.sum(F.col("Sales")).alias("CitySales")).orderBy(F.desc(F.col("CitySales"))).show(10)

# Tabla de número de pedidos e importe por horas (campo hour extraído de Order Date).
dfventas.groupBy(F.hour(F.col("timestamp")).alias("Hour")) \
    .agg(F.sum(F.col("Quantity Ordered")).alias("TotalQty"),F.sum(F.col("Sales"))) \
    .orderBy(F.col("Hour")).show(24)



In [0]:
# Opciones clave por fuente

# Archivos

df = spark.readStream.format("csv") \
    .option("path", "/ruta/al/directorio") \
    .option("maxFilesPerTrigger", 1) \  # Nº de archivos leídos por trigger
    .option("latestFirst", True) \      # Leer los archivos más recientes primero
    .load()


In [0]:
# Socket: sólo pruebas, sin tolerancia a fallos

df = spark.readStream.format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()


In [0]:
# Kafka

df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "host1:port1") \
    .option("subscribe", "topic_name") \
    .option("startingOffsets", "latest") \  # "earliest" para leer desde el inicio
    .load()


In [0]:
# Rate: genera datos automáticamente, con un timestamp y value creciente

df = spark.readStream.format("rate") \
    .option("rowsPerSecond", 10) \
    .load()


In [0]:
# Tablas: Útil para persistencia y consultas en tiempo real.

df = spark.readStream.table("nombre_tabla")


## Modos de salida (outputMode)

Cuando escribimos un DataFrame en streaming (writeStream), debemos especificar cómo se actualizan los datos en el destino:

| Modo      | Descripción                                                                 |
|-----------|-----------------------------------------------------------------------------|
| append    | Solo agrega nuevas filas al destino. No admite actualizaciones o eliminaciones. |
| complete  | Escribe todos los resultados en cada trigger. Útil para agregaciones.       |
| update    | Similar a append, pero actualiza filas ya procesadas. Necesario cuando se usan operaciones de estado (como agregaciones incrementales). |

## Triggers (trigger)

Define cada cuánto tiempo se procesan los datos:

| Trigger                              | Descripción                                                      |
|--------------------------------------|------------------------------------------------------------------|
| trigger(processingTime="10 seconds") | Ejecuta la consulta cada 10 segundos.                           |
| trigger(once=True)                  | Ejecuta la consulta solo una vez, procesando todos los datos disponibles. |
| trigger(continuous="1 second")      | Para baja latencia (solo con fuentes compatibles como Kafka).   |

## Destinos (sinks)

El sink define dónde se escriben los datos procesados:

| Sink       | Descripción                                                        |
|------------|--------------------------------------------------------------------|
| console    | Muestra los resultados en la consola (útil para pruebas).          |
| memory     | Guarda en memoria (puede ser consultado con SQL).                  |
| file       | Escribe en formato JSON, Parquet, CSV, etc.                       |
| kafka      | Escribe mensajes en un topic de Apache Kafka.                     |
| foreach    | Permite escribir en un destino personalizado (API de Python/Java). |

Tolerancia a fallos y Checkpoints
Spark Streaming garantiza tolerancia a fallos con Checkpoints y WAL (Write-Ahead Logs).

Checkpointing: Almacena el estado del streaming en un directorio para recuperarlo en caso de fallo.
Habilitación de checkpoints:
python
Copiar
Editar
query = df.writeStream \
    .format("parquet") \
    .option("path", "/ruta/salida") \
    .option("checkpointLocation", "/ruta/checkpoints") \
    .start()
Esto permite recuperación automática en caso de fallo del nodo o reinicio de la aplicación.

📆 Windowing (Ventanas temporales)
Permite agrupar datos en intervalos de tiempo en lugar de procesar cada evento individualmente.

Ejemplo: Contar eventos en ventanas de 10 segundos con 5 segundos de solapamiento:

python
Copiar
Editar
from pyspark.sql.functions import window

df_window = df.groupBy(window("timestamp", "10 seconds", "5 seconds")).count()
🔹 Útil en casos como:

Contar palabras en un flujo de datos por minuto.
Agregaciones en tiempo real con datos que llegan en diferentes momentos.
💧 Watermarking
Controla la cantidad de datos antiguos que se retienen en memoria, permitiendo manejar retrasos en la llegada de datos sin consumir demasiados recursos.

Ejemplo: Mantener datos de hasta 10 minutos en un groupBy basado en ventana de tiempo:

python
Copiar
Editar
df_watermarked = df \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(window("timestamp", "5 minutes")) \
    .count()
🔹 Evita acumulación de estado indefinida y mejora la escalabilidad del sistema.

🔗 Joins en Streaming
Se pueden realizar uniones entre:

Streaming vs Batch ✅
Streaming vs Streaming (con restricciones) 🚧
Ejemplo de join entre un flujo de datos y una tabla estática:

python
Copiar
Editar
df_streaming.join(df_estatico, "id", "inner")
Para unir dos streams, es necesario que al menos uno tenga Watermark:

python
Copiar
Editar
df1.withWatermark("timestamp", "10 minutes") \
    .join(df2.withWatermark("timestamp", "10 minutes"), "id", "inner")
🔹 Uso común en correlación de eventos, como unir registros de compras en streaming con datos de clientes en batch.

